# Combining cities and states using +

In this lesson, you learned how to combine columns into one.

The `clients` table has one column, `city`, to store the cities where the clients live, and another column, `state`, to store the state of the city.
```
| client_id | client_name | client_surname | city      | state    |
|-----------|-------------|----------------|-----------|----------|
| 1         | Miriam      | Antona         | Las Vegas | Nevada   |
| 2         | Astrid      | Harper         | Chicago   | Illinois |
| 3         | David       | Madden         | Phoenix   | Arizona  |
| ...       | ...         | ...            | ...       | ...      |
```
You need to combine `city` and `state` columns into one, to have the following format: `'Las Vegas, Nevada'`.

You will use `+` operator to do it.

```
SELECT 
	client_name,
	client_surname,
    -- Concatenate city with state
    city + ', ' + state AS city_state
FROM clients
```

```
SELECT 
	client_name,
	client_surname,
    -- Consider the NULL values
	ISNULL(city, '') + ISNULL(', ' + state, '') AS city_state
FROM clients
```

# Concatenating cities and states

In the previous exercise, you used the `+` operator to combine two columns into one. This time you need to do the same, but using the `CONCAT()` function.

To get this format for the concatenation: `'Las Vegas, Nevada'`, you will need to use the `CASE` expression in case the `state` column has `NULL` values.

```
SELECT 
		client_name,
		client_surname,
    -- Use the function to concatenate the city and the state
		CONCAT(
				city,
				CASE WHEN state IS NULL THEN '' 
				ELSE CONCAT(', ', state) END) AS city_state
FROM clients
```

# Working with DATEFROMPARTS()

In this lesson, you also learned how to combine different parts of a date, which are in separate columns into one.

In the `paper_shop_daily_sales` table, the columns `year_of_sale`, `month_of_sale`, and `day_of_sale`, store the different values of a date.
```
| product_name | units | year_of_sale | month_of_sale | day_of_sale |
|--------------|-------|--------------|---------------|-------------|
| notebooks    | 2     | 2019         | 1             | 1           |
| notebooks    | 3     | 2019         | 5             | 12          |
| ...          | ...   | ...          | ...           | ...         |
```
You need to combine all these columns into one, by using the `DATEFROMPARTS()` function.

```
SELECT 
	product_name,
	units,
    -- Use the function to concatenate the different parts of the date
	DATEFROMPARTS(
      	year_of_sale, 
      	month_of_sale, 
      	day_of_sale) AS complete_date
FROM paper_shop_daily_sales
```

Which of the following statements about `DATEFROMPARTS()` is true?
- If one argument passed to `DATEFROMPARTS()` has a `NULL` value, it will return `NULL`.

# Using SUBSTRING() and CHARINDEX()

In this lesson, you learned how to split one column into more columns.

The `clients_split` table has one column, `city_state`, that stores the cities where the clients live and the state of the city. The values of this column have this appearance: `'Chicago, Illinois'`.

You need to split this column into two new columns, one for the city and the other one for the state. You think about using `SUBSTRING()` in combination with `CHARINDEX()` and `LEN()`.

```
SELECT 
	client_name,
	client_surname,
    -- Extract the name of the city
	SUBSTRING(city_state, 1, CHARINDEX(', ', city_state) - 1) AS city,
    -- Extract the name of the state
    SUBSTRING(city_state, CHARINDEX(', ', city_state) + 1, LEN(city_state)) AS state
FROM clients_split
```

# Using RIGHT() , LEFT() and REVERSE()

In the previous exercise, you used `SUBSTRING()` and `CHARINDEX()` to split the `city_state` column into two new columns.

This time you need to do the same, but using the `LEFT()`, `RIGHT()`, and `REVERSE()` functions.

```
SELECT
	client_name,
	client_surname,
    -- Extract the name of the city
	LEFT(city_state, CHARINDEX(', ', city_state) - 1) AS city,
    -- Extract the name of the state
    RIGHT(city_state, CHARINDEX(' ,', REVERSE(city_state)) - 1) AS state
FROM clients_split
```

# SUBSTRING() or CHARINDEX()?

In this lesson, you studied how to use the `SUBSTRING()` and `CHARINDEX()` functions to split one column into two new columns.

Which of the following statements about these functions is false?

- `SUBSTRING()` returns the position of a substring in a string. We can optionally determine the position where the search will start.

# PIVOT or UNPIVOT?

In this lesson, you learned the differences between using `PIVOT` and `UNPIVOT`.

Which of the following statements about these relational operators is false?

- `UNPIVOT` turns the unique values from one column into multiple columns in the output.

# Turning rows into columns

In this lesson, you learned that `PIVOT` turns the unique values from one column into multiple columns.

Analyzing the data of `paper_shop_monthly_sales`, you realize the structure of this table is not appropriate for the report that you want to make.

You want to generate a report with this appearance:
```
|year_of_sale|notebooks|pencils|crayons|
|------------|---------|-------|-------|
| 2018       | 150     | 150   | 80    |
| 2019       | 230     | 130   | 170   |
```
In other words, you want to change the data you have in the rows to data into columns, and sum the units for every year.

As you learned from the previous exercises, the name of the products and the units has to be split. This is done in the subselect, take a look at it.

```
SELECT
	year_of_sale,
    -- Select the pivoted columns
	notebooks, 
	pencils, 
	crayons
FROM
   (SELECT 
		SUBSTRING(product_name_units, 1, charindex('-', product_name_units)-1) product_name, 
		CAST(SUBSTRING(product_name_units, charindex('-', product_name_units)+1, len(product_name_units)) AS INT) units,	
		year_of_sale
	FROM paper_shop_monthly_sales) sales
-- Sum the units for column that contains the values that will be column headers
PIVOT (SUM(units) FOR product_name IN (notebooks, pencils, crayons))
-- Give the alias name
AS paper_shop_pivot
```

# Turning columns into rows

In the previous exercise, you turned the names of the products you had in the rows into columns, and then you summarized the units of the products for every year.

Suppose you stored the result from the previous exercise in a new table called `pivot_sales`, and now you want to turn the columns `notebooks`, `pencils`, and `crayons` into row values.

The expected result will be:
```
| year_of_sale | units | product_name |
|--------------|-------|--------------|
| 2018         | 150   | notebooks    |
| 2018         | 150   | pencils      |
| 2018         | 80    | crayons      |
| 2019         | 230   | notebooks    |
| 2019         | 130   | pencils      |
| 2019         | 170   | crayons      |
```

```
SELECT * FROM pivot_sales
-- Use the operator to convert columns into rows
UNPIVOT
	-- The resulting column that will contain the turned columns into rows
	(units FOR product_name  IN (notebooks, pencils, crayons))
-- Give the alias name
AS unpivot_sales
```